In [ ]:
import numpy as np
import pandas as pd
from pathlib import Path

import dascore as dc

from das_anomaly.settings import SETTINGS

In [ ]:
txt_path  = SETTINGS.RESULTS_PATH

waterfall_path = "path/for/waterfall/plots"
waterfall_dir = Path(waterfall_path)
waterfall_dir.mkdir(parents=True, exist_ok=True)  # creates it if it doesn't exist

data_path = SETTINGS.DATA_PATH
data_unit = SETTINGS.DATA_UNIT

min_freq = SETTINGS.MIN_FREQ 
max_freq = SETTINGS.MAX_FREQ 
step_multiple = SETTINGS.STEP_MULTIPLE 
start_channel = SETTINGS.START_CHANNEL 
end_channel = SETTINGS.END_CHANNEL 
time_window = SETTINGS.TIME_WINDOW
time_overlap = SETTINGS.TIME_OVERLAP 

In [ ]:
# Read one path per line
s = pd.read_csv(txt_path, header=None, names=["path"], dtype=str)

# Extract timestamps
ts = s["path"].str.extract(
    r'(?P<time_min>\d{4}_\d{2}_\d{2}T\d{2}_\d{2}_\d{2})__'
    r'(?P<time_max>\d{4}_\d{2}_\d{2}T\d{2}_\d{2}_\d{2})'
)

# Build DataFrame with 1-based row numbers
df = pd.concat([s.index.to_series().add(1).rename("rows"), ts], axis=1)

print(df.head())

In [ ]:
def _distance_slice(patch):
    """Convert channel range to distance range."""
    start = patch.coords["distance"][start_channel]
    end = patch.coords["distance"][end_channel]
    return (start, end)


In [ ]:
sp = dc.spool(data_path)


In [ ]:

select_time = True

for row in df.itertuples(index=True):  # index=False if you don't need the index
    i = row.Index          # row number (DataFrame index)
    t1 = pd.to_datetime(row.time_min, format="%Y_%m_%dT%H_%M_%S")       
    t2 = pd.to_datetime(row.time_max, format="%Y_%m_%dT%H_%M_%S")
    
    if i>1:
         break

    if select_time:
        sub_sp_time = sp.select(time=(t1, t2))
        sub_sp_time_distance = sub_sp_time.select(distance=(_distance_slice(sub_sp_time[0])))
        sub_sp = sub_sp_time_distance
    else:
        sub_sp_distance = sp.select(distance=(_distance_slice(sp[0]))) 
        sub_sp = sub_sp_distance
    # chunk into windowed sub-patches
    sub_sp_chunked = sub_sp.sort("time").chunk(
        time=time_window, keep_partial=True,
    )
    if len(sub_sp_chunked) == 0:
        raise ValueError("No patch of DAS data found within data path: %s")
    # iterate over patches and perform preprocessing
    for pa_num, patch in enumerate(sub_sp_chunked):
        if data_unit == "velocity":
                proc_patch = patch.velocity_to_strain_rate_edgeless(
                    step_multiple=step_multiple
                ).detrend("time")
        elif data_unit == "strain_rate":
                proc_patch = patch.detrend("time")
        else:
            raise ValueError(
                "Unsupported data_unit: {data_unit!r}. "
                "Expected 'velocity' or 'strain_rate'."
            )
        ax = proc_patch.viz.waterfall(show=False, scale=(-2.5e-6,2.5e-6))
        fig = ax.get_figure()          # or: fig = ax.figure
        name = Path(s["path"].iloc[i]).stem
        fig.savefig(waterfall_dir / f"{name}_{pa_num}.png", dpi=300, bbox_inches="tight")